## FII_SMART Multi-evidence Natural Language Inference for Clinical Trial Data


In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb 12 14:19:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import sys
import os

DEBUG1=False
DEBUG2=False

PREFIXG="/content/"
PREFIX="/content/"

TRAIN_DATA=PREFIXG+"train.csv"
TEST_DATA=PREFIXG+"dev.csv"
TEST_DATA2=PREFIXG+"test.csv"

Clean files for new run

In [4]:
from google.colab import files

!ls
!rm test.*
!rm train.*
!rm dev.*
!rm -rf CT_data
!rm CT_data.zip
!rm dataset.zip

sample_data
rm: cannot remove 'test.*': No such file or directory
rm: cannot remove 'train.*': No such file or directory
rm: cannot remove 'dev.*': No such file or directory
rm: cannot remove 'CT_data.zip': No such file or directory
rm: cannot remove 'dataset.zip': No such file or directory


In [5]:
!pip install transformers
!wget https://github.com/volosincu/FII_Smart__Semeval2023/raw/main/dataset.zip
!unzip /content/dataset.zip
!unzip /content/CT_data.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.9 MB/s eta 0:00:00
--2023-02-12 14:19:12--  https://github.com/volosincu/FII_Smart__Semeval2023/raw/main/dataset.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/volosincu/FII_Smart__Semeval2023/main/dataset.zip [following]
--2023-02-12 14:19:13--  https://raw.githubusercontent.com/volosincu/FII_Smart__Semeval2023/main/dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserc

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))

STOP_WORDS_RO = ['e.g.', '•','â•','mm¬', 'mm¬¨','trial','study','patient', 'patients', 'candidate', 'candidates', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'a', 'an', 'the', 'here', 'there', 'why', 'how', 'so', 's', 't', 'can', 'will', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ma', ]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:

punctuation = ['„', '”', '(', ')', ';', '{', '}','[',']', '/', '\\',"\"", "'", ','] 
def normalizeChunk(chunk):
  _word = chunk.lower()
  word = None
  withSeparator = False;
  if _word.startswith('('):
    word = _word[1:]
  elif _word.endswith(')'):
    word = _word[:len(_word)-1]
  # elif _word.endswith(','):
  #   word = _word[:len(_word)-1]
  # elif _word.endswith('.'):
  #   # withSeparator = True
  #   word = _word[:len(_word)-1]
  elif _word.endswith(':'):
    word = _word[:len(_word)-1]
  elif _word.endswith(';'):
    word = _word[:len(_word)-1]
  elif _word.startswith('„'):
    word = _word[1:]
  elif _word.endswith('”'):
    word = _word[:len(_word)-1]
  elif _word.startswith('{'):
    word = _word[1:]
  elif _word.startswith('['):
    word = _word[1:]
  elif _word.startswith("\'"):
    word = _word[1:]
  elif _word.endswith('\''):
    word = _word[:len(_word)-1]
  elif _word.endswith('}'):
    word = _word[:len(_word)-1]
  elif _word.endswith(']'):
    word = _word[:len(_word)-1]
  else:
    word = _word

  concats=[]
  for sign in punctuation:
    if(word.__contains__(sign)):
      c = word.split(sign)
      for w in c:
        sw = normalizeChunk(w)
        for w1 in sw:
          if(not concats.__contains__(sw)):
            concats.append(w1)
  if(len(concats) == 0):
    concats.append(word)
    if(withSeparator):
      concats.append("<START>")
  return list(dict.fromkeys(concats))

normalizeChunk("casa.„Toata")

['casa.', 'toata']

In [8]:
def normalizeDocument(text, removeStopWords):
  NORMALIZED_CORPUS = []
  for _word in text.split():
    if(_word.__contains__('>=')):
      _word = _word.replace(">=", "greater than or equal ")
      #print(_word)
    if(_word.__contains__('=<')):
      _word = _word.replace("=<", "smaller than or equal ")
      #print(_word)
    if(_word.__contains__('<')):
      _word = _word.replace("<", "smaller than ")
      #print(_word)
    if(_word.__contains__('>')):
      _word = _word.replace(">", "greater than ")
      #print(_word)
    normalized = normalizeChunk(_word)
    for word in normalized:
      if(removeStopWords):
        if(not STOP_WORDS_RO.__contains__(word)):
          NORMALIZED_CORPUS.append(word)
      else:
        NORMALIZED_CORPUS.append(word)
  return " ".join(NORMALIZED_CORPUS)

normalizat = normalizeDocument("Renal function: Creatinine =< 2.0 mg/dl or a creatinine clearance >= 50 mg/min", True)
normalizat

'renal function creatinine smaller than or equal  2.0 mg dl or creatinine clearance greater than or equal  50 mg min'

In [9]:
import uuid
import json
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import pandas as pd


from transformers import AutoTokenizer, AutoModel


from datetime import datetime
current_dateTime = datetime.now()

In [10]:
def stamp_time():
    return "".join("".join("".join(str(current_dateTime).split(".")[0].split(":")).split("-")).split(" "))


In [11]:
OVF=0
OVF_V=[]

In [12]:
VECTOR_MAX=100

EPOCHS = 2
LAYER_1_NODES=256
LAYER_2_NODES=2
LAYER_1_ACTIVATION='relu'
LAYER_2_ACTIVATION='softmax'
LOSS_FN='sparse_categorical_crossentropy'
#LOSS_FN='binary_crossentropy'
METRIC='accuracy'
FIT_BATCH_SIZE=16

In [13]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
model_name = 'giacomomiolo/biobert_reupload'
# model_name = 'dmis-lab/biobert-base-cased-v1.1'
# model_name = 'bert-base-multilingual-cased'
# model_name = "allenai/scibert_scivocab_uncased"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

## Data collection utils


In [14]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)


def collectEvidenceData(evidenceSection, evidenceIndexes, clinicalTrialId, evidenceSectionId, DEBUG2):
    evidences = []
    for evIndex in evidenceIndexes:
        try:
            i = int(evIndex)
            if DEBUG2:
                print(evidenceSection[i])
            norm_evid = normalizeDocument(evidenceSection[i].strip(), True)
            evidences.append(norm_evid)
        except:
            print(evidenceIndexes)
            print('\n!!! Failed to collect evidence on index, trialId, section: ' + str(i) +' '+ clinicalTrialId, evidenceSectionId)
    return evidences


def parseEvidenceSingle(hypotesis, clinicalTrialId, evidenceSectionId, primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2):
    evidences = []
    if DEBUG1:
        print("Parse evidence Single: " + clinicalTrialId + " " + evidenceSectionId)
    CT_PATH = PREFIX+'CT_data/'+clinicalTrialId+'.json'
    try:
        with open(CT_PATH) as json_file:
            dataCT = json.load(json_file)
            if DEBUG2:
                print(primaryEvidenceIndexes)
        evidences = collectEvidenceData(dataCT[evidenceSectionId], primaryEvidenceIndexes, clinicalTrialId, evidenceSectionId, False)
        if DEBUG2:
            print(evidences)
        tok = encode_sentence("[CLS] "+hypotesis+" [SEP] " + " ".join(evidences))
        # print("[CLS] "+hypotesis+" [SEP] " + " ".join(evidences))
        # print("       aa          ")
        # if (len(tok)>300):
        #     # OVF=OVF+1
        #     OVF_V.append(len(tok))
        #     print("!!!!!!!!!!!!!!!!")
        #     print(hypotesis)
        #     print(clinicalTrialId)

    except Exception as err:
        print('\n!!! Error for file: ' + clinicalTrialId)
        print(err)
    if DEBUG2:
        print("\n")
    return evidences

In [15]:
def splitEvidence(acc, tmp, hypotesis, evidences, evIndex, DEBUG):
  if (len(evidences)==0):
    return acc
  if (evIndex>len(evidences)-1):
    acc.append(tmp)
    # print("----------1-----------EXIT split evidenta ---------------------")
    return acc;
  # print(evidences)
  # print(evIndex)
  hypTmpl = "[CLS] "+hypotesis+" [SEP] "
  tok = encode_sentence(hypTmpl + tmp + evidences[evIndex]+" ")
  if(len(tok)<VECTOR_MAX):
    tmp = tmp + evidences[evIndex]+" "
    splitEvidence(acc, tmp, hypotesis, evidences, evIndex+1, DEBUG)
  else:
    acc.append(tmp)
    tmp=""
    tok = encode_sentence(hypTmpl + tmp + evidences[evIndex]+" ")
    if(len(tok)<VECTOR_MAX):
      if (evIndex<len(evidences)):
        tmp = tmp + evidences[evIndex]
        splitEvidence(acc, tmp, hypotesis, evidences, evIndex+1, DEBUG)
      else:
        # print("----------2-----------EXIT split evidenta ---------------------")
        acc.append(evidences[evIndex])
        return acc
    else:
      if (DEBUG):
        print("!!!!!!!! hypotesis + evidence > " + str(VECTOR_MAX) + "!!!!!!!!!!")
      return acc
  return acc


In [16]:
STM = pd.read_csv(TRAIN_DATA) #, nrows=40
print(STM.head(5))
print(STM.shape)

# exit()
_EVIDENCES=[]
_HYPOTESIS=[]
Y_LABELS=np.array([], dtype="int32")
CT_index = -1
for id in STM['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = STM['Primary_id'][CT_index]
    hypotesis = normalizeDocument(STM['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    if(hypotesisLen > 90):
      print('%%%% ')
    if (STM['Type'][CT_index] == "Single"):
      primaryEvidenceIndexes = STM['Primary_evidence_index'][CT_index].split(',')
      evidences = parseEvidenceSingle(hypotesis, CTid, STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", hypotesis, evidences, 0, DEBUG2)
      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          _EVIDENCES.append(evidenceFragment)
          _HYPOTESIS.append(hypotesis)
          if(STM['Label'][CT_index] == 'Entailment'):
            Y_LABELS = np.append(Y_LABELS, [1], axis=0)
          else:
            Y_LABELS = np.append(Y_LABELS, [0], axis=0)
          if (CT_index < 17):
            print("     - "+ evidenceFragment)
      
      # if (CT_index == 3):
      #     exit()
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")
    if (STM['Type'][CT_index] == "Comparison"):
      CTidSecondary = STM['Secondary_id'][CT_index]
      primaryEvidenceIndexes = STM['Primary_evidence_index'][CT_index].split(',')
      secondaryEvidenceIndexes = STM['Secondary_evidence_index'][CT_index].split(',')
      premisesPrimary = parseEvidenceSingle(hypotesis, CTid, STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvidenceSingle(hypotesis, CTidSecondary, STM['Section_id'][CT_index], secondaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)

      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          _EVIDENCES.append(evidenceFragment)
          _HYPOTESIS.append(hypotesis)
          if(STM['Label'][CT_index] == 'Entailment'):
            Y_LABELS = np.append(Y_LABELS, [1], axis=0)
          else:
            Y_LABELS = np.append(Y_LABELS, [0], axis=0)
        if (CT_index < 17):
          print("     - "+ evidenceFragment)
        
      # if (CT_index == 3):
      #     exit()
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")




Y_LABELS

                                     Id        Type      Section_id  \
0  5bc844fc-e852-4270-bfaf-36ea9eface3d  Comparison    Intervention   
1  86b7cb3d-6186-4a04-9aa6-b174ab764eed      Single     Eligibility   
2  dbed5471-c2fc-45b5-b26f-430c9fa37a37  Comparison  Adverse Events   
3  20c35c89-8d23-4be3-b603-ac0ee0f3b4de      Single     Eligibility   
4  f17cb242-419d-4f5d-bfa4-41494ed5ac0e  Comparison    Intervention   

    Primary_id Secondary_id  \
0  NCT01928186  NCT00684983   
1  NCT00662129          NaN   
2  NCT00093145  NCT00703326   
3  NCT01097642          NaN   
4  NCT00852930  NCT02308020   

                                           Statement          Label  \
0  All the primary trial participants do not rece...  Contradiction   
1  Patients with Platelet count over 100,000/mm¬¨...  Contradiction   
2  Heart-related adverse events were recorded in ...     Entailment   
3  Adult Patients with histologic confirmation of...  Contradiction   
4  Laser Therapy is in each coh

array([0, 0, 0, ..., 1, 1, 0])

## Encoding

In [17]:
def encode_data_to_tensors(statements, evidences, tokenizer, dim):
    
    tokStm = [encode_sentence(s)
        for s in statements]

    tokPrem = [encode_sentence(s)
        for s in evidences]
    tokZip = list(zip(tokStm, tokPrem))

    # tokIntersect = [np.where(np.in1d(xt[0], np.intersect1d(xt[0], xt[1])), np.zeros_like(xt[0]), [1])
    #     for xt in tokZip]
    # type_s1 = tf.ragged.constant(tokIntersect);

    # tokIntersect = [np.where(np.in1d(xt[1], np.intersect1d(xt[0], xt[1])), np.zeros_like(xt[1]), [1])
    #     for xt in tokZip]
    # type_s2 = tf.ragged.constant(tokIntersect);

    statementsTokenizedTensor = tf.ragged.constant(tokStm)
    evidencesTokenizedTensor = tf.ragged.constant(tokPrem)


    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*statementsTokenizedTensor.shape[0]
    _word_ids = tf.concat([cls, statementsTokenizedTensor, evidencesTokenizedTensor], axis=-1)
    ii = 0
    max = 0
    for ip in _word_ids:
        ii = ii + 1
        if (ip.shape[0] >= max):
            max = ip.shape[0]
            # print(ii)
            # print(max)
    print("Dataset MAX (dim): ")
    print(max)

    # Padding - adaug 0 sau 1 la tensori pt a avea aceiasi dimensiune 
    # paggings [ top bottom ] [left right] - adaug doar la final
    paddings = tf.constant([[0, 0,], [0, dim-max]])
    input_word_ids = tf.pad(_word_ids.to_tensor(), paddings, "CONSTANT", constant_values=0)
    input_mask = tf.ones_like(input_word_ids)

    type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(statementsTokenizedTensor)
    type_s2 = tf.ones_like(evidencesTokenizedTensor)
    input_type_ids = tf.pad(tf.concat(
        [type_cls, type_s1, type_s2], axis=-1).to_tensor(), paddings, "CONSTANT", constant_values=0)

    inputs = {
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': input_type_ids}

    return inputs


## Model declaration

In [18]:
def build_model():
    bert_encoder = TFBertModel.from_pretrained(model_name)
    # bert_encoder = AutoModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(VECTOR_MAX,), dtype=tf.int32, name="input_type_ids")
    # print(input_word_ids)

    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]

    # lstm = tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)(embedding[:,0,:])
    x = tf.keras.layers.Dense(256, activation=LAYER_1_ACTIVATION)(embedding[:,0,:])
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(1024, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(2048, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(256, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.9)(x)
    x = tf.keras.layers.Dense(128, activation=LAYER_1_ACTIVATION)(x)
    x = tf.keras.layers.LeakyReLU(0.7)(x)
    output = tf.keras.layers.Dense(2, activation=LAYER_2_ACTIVATION)(x)

    learning_rate = 2e-5


    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=learning_rate/(EPOCHS*0.15)), loss=LOSS_FN, metrics=[METRIC])  # decay=learning_rate/(epochs*0.5)

    return model


# Fit and Run model

In [19]:
train_input = encode_data_to_tensors(_HYPOTESIS, _EVIDENCES, tokenizer, VECTOR_MAX)
# check shape data 
print('DEBUG train shapes: ')
print(train_input['input_word_ids'].shape)
print(train_input['input_mask'].shape)
print(train_input['input_type_ids'].shape)

Dataset MAX (dim): 
99
DEBUG train shapes: 
(3880, 100)
(3880, 100)
(3880, 100)


In [20]:
model = build_model()
df = pd.DataFrame()
df['Y'] = Y_LABELS


print("dim ipoteze:")
print(len(_HYPOTESIS))
print("dim evidențe:")
print(len(_EVIDENCES))
print("dim Y:")
print(len(Y_LABELS))

model.fit(x=train_input, y=Y_LABELS, epochs = 7, verbose = 1, batch_size = FIT_BATCH_SIZE, validation_split = 0.4)

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at giacomomiolo/biobert_reupload.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


dim ipoteze:
3880
dim evidențe:
3880
dim Y:
3880
Epoch 1/7


146/146 [==============================] - 71s 390ms/step - loss: 0.6937 - accuracy: 0.5185 - val_loss: 0.7010 - val_accuracy: 0.4659
Epoch 2/7
146/146 [==============================] - 57s 393ms/step - loss: 0.6666 - accuracy: 0.5795 - val_loss: 0.7798 - val_accuracy: 0.4575
Epoch 3/7
146/146 [==============================] - 59s 404ms/step - loss: 0.5166 - accuracy: 0.7401 - val_loss: 1.1436 - val_accuracy: 0.3763
Epoch 4/7
146/146 [==============================] - 60s 411ms/step - loss: 0.3261 - accuracy: 0.8501 - val_loss: 1.7442 - val_accuracy: 0.3576
Epoch 5/7
146/146 [==============================] - 61s 416ms/step - loss: 0.1992 - accuracy: 0.9128 - val_loss: 2.3528 - val_accuracy: 0.3505
Epoch 6/7
146/146 [==============================] - 61s 420ms/step - loss: 0.1393 - accuracy: 0.9433 - val_loss: 2.4267 - val_accuracy: 0.3570
Epoch 7/7
146/146 [==============================] - 62s 424ms/step - loss: 0.0951 - accuracy: 0.9588 - val_loss: 3.3171 - val_accuracy: 0.3686


## Evaluare, test data


In [21]:
TEST_STM = pd.read_csv(TEST_DATA) #, nrows=40
print(TEST_STM.head(5))
print(TEST_STM.shape)


# exit()
TEST_EVIDENCES=[]
TEST_HYPOTESIS=[]
TEST_UUID=[]
TEST_LABEL=[]
TEST_Y_LABELS=np.array([], dtype="int32")
CT_index = -1
for id in TEST_STM['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = TEST_STM['Primary_id'][CT_index]
    hypotesis = normalizeDocument(TEST_STM['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    if (TEST_STM['Type'][CT_index] == "Single"):
      primaryEvidenceIndexes = TEST_STM['Primary_evidence_index'][CT_index].split(',')
      evidences = parseEvidenceSingle(hypotesis, CTid, TEST_STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", "[CLS] "+hypotesis+" [SEP] ", evidences, 0, DEBUG2)
      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
          TEST_LABEL.append(TEST_STM['Label'][CT_index])
          if(TEST_STM['Label'][CT_index] == 'Entailment'):
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [1], axis=0)
          else:
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [0], axis=0)
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")
    if (TEST_STM['Type'][CT_index] == "Comparison"):
      CTidSecondary = TEST_STM['Secondary_id'][CT_index]
      primaryEvidenceIndexes = TEST_STM['Primary_evidence_index'][CT_index].split(',')
      secondaryEvidenceIndexes = TEST_STM['Secondary_evidence_index'][CT_index].split(',')
      premisesPrimary = parseEvidenceSingle(hypotesis, CTid, TEST_STM['Section_id'][CT_index], primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvidenceSingle(hypotesis, CTidSecondary, TEST_STM['Section_id'][CT_index], secondaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)
      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
          TEST_LABEL.append(TEST_STM['Label'][CT_index])
          if(TEST_STM['Label'][CT_index] == 'Entailment'):
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [1], axis=0)
          else:
            TEST_Y_LABELS = np.append(TEST_Y_LABELS, [0], axis=0)
        if (CT_index < 17):
          print("     - "+ evidenceFragment)
      # print("-------2--------")
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")


print("dim ipoteze:")
print(len(TEST_EVIDENCES))
print("dim evidențe:")
print(len(TEST_EVIDENCES))
print("dim Y:")
print(len(TEST_Y_LABELS))

                                     Id        Type      Section_id  \
0  1adc970c-d433-44d0-aa09-d3834986f7a2      Single         Results   
1  6b9162d0-0816-46d4-81af-c60028dcc63b  Comparison     Eligibility   
2  0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f  Comparison  Adverse Events   
3  cc1f712a-2116-4e40-9810-f315e3fa5ff8      Single         Results   
4  904061c0-14fa-4f13-9118-9a41e24fa8eb      Single     Eligibility   

    Primary_id Secondary_id  \
0  NCT00066573          NaN   
1  NCT00425854  NCT01224678   
2  NCT02273973  NCT00281697   
3  NCT00593346          NaN   
4  NCT02340221          NaN   

                                           Statement          Label  \
0  there is a 13.2% difference between the result...  Contradiction   
1  Patients with significantly elevated ejection ...  Contradiction   
2  a significant number of the participants in th...  Contradiction   
3  the primary trial does not report the PFS or o...     Entailment   
4  Prior treatment with fulvest

In [22]:
test_input = encode_data_to_tensors(TEST_HYPOTESIS, TEST_EVIDENCES, tokenizer, VECTOR_MAX)

# check shape data 
print('DEBUG test shapes: ')
print(test_input['input_word_ids'].shape)
print(test_input['input_mask'].shape)
print(test_input['input_type_ids'].shape)

Dataset MAX (dim): 
99
DEBUG test shapes: 
(386, 100)
(386, 100)
(386, 100)


In [23]:
predictions = [np.argmax(i) for i in model.predict(test_input)]
# print([i for i in model.predict(test_input)])

submission = pd.DataFrame()
submission['UUID'] = TEST_UUID
submission['label'] = TEST_LABEL
submission['prediction'] = predictions

RESULTS_FILE = PREFIXG+"/submission-"+stamp_time()+".csv"
RESULTS_FILE_R = PREFIXG+"/results-"+stamp_time()+".json"
submission.to_csv(RESULTS_FILE, index = False)
submission.head()

13/13 [==============================] - 5s 197ms/step


,UUID,label,prediction
0,1adc970c-d433-44d0-aa09-d3834986f7a2,Contradiction,1
1,1adc970c-d433-44d0-aa09-d3834986f7a2,Contradiction,1
2,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,0
3,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,0
4,6b9162d0-0816-46d4-81af-c60028dcc63b,Contradiction,0


# Postprocessing

Conversion from json to csv of training, test and dev files. 

In [24]:
import json
import pandas as pd


"""
type = train | dev
"""
def preprocessing(JSONPATH, type, PREFIX):

    with open(JSONPATH) as json_file:
        data = json.load(json_file)

    columns = ['Type', 'Section_id', 'Primary_id', 'Secondary_id', 'Statement', 'Label', ] #  'Primary_evidence_index', 'Secondary_evidence_index',

    ids = []
    types = []
    sectionId = []
    primaryId = []
    secondaryId = []
    statement = []
    label = []
    PEI = []
    SEI = []
    for entry in data:
        ids.append(entry)
        types.append(data[entry]['Type'])
        sectionId.append(data[entry]['Section_id'])
        primaryId.append(data[entry]['Primary_id'])
        if not (data[entry].get('Secondary_id') is None):
            secondaryId.append(data[entry]['Secondary_id'])
        else:
            secondaryId.append(None)
        statement.append(data[entry]['Statement'])
        label.append(data[entry]['Label'])
        PEI.append(','.join(str(x) for x in data[entry]['Primary_evidence_index']))
        if not (data[entry].get('Secondary_evidence_index') is None):
            SEI.append(','.join(str(x) for x in data[entry]['Secondary_evidence_index']))
        else:
            SEI.append(','.join(str(x) for x in []))

    df = pd.DataFrame()
    df['Id'] = ids
    df['Type'] = types
    df['Section_id'] = sectionId
    df['Primary_id'] = primaryId
    df['Secondary_id'] = secondaryId
    df['Statement'] = statement
    df['Label'] = label
    df['Primary_evidence_index'] = PEI
    df['Secondary_evidence_index'] = SEI

    #print(df.head(20))

    compression_opts = dict(method='zip', archive_name='out.csv') 
    # df.to_csv('out.zip', index=False, compression=compression_opts) 
    df.to_csv(PREFIX+type+'.csv', index=False) 



def collectEvidenceData(evidenceSection, evidenceIndexes, clinicalTrialId, evidenceSectionId, DEBUG2):
    evidences = []
    for evIndex in evidenceIndexes:
        try:
            i = int(evIndex)
            if DEBUG2:
                print(evidenceSection[i])
            evidences.append(evidenceSection[i].strip())
        except:
            print(evidenceIndexes)
            print('\n!!! Failed to collect evidence on index, trialId, section: ' + str(i) + clinicalTrialId, evidenceSectionId)
    return evidences


def parseEvidenceSingle(clinicalTrialId, evidenceSectionId, primaryEvidenceIndexes, PREFIX, DEBUG1, DEBUG2):
    if DEBUG1:
        print("Parse evidence Single: " + clinicalTrialId + " " + evidenceSectionId)
    CT_PATH = PREFIX+'/CT_data/'+clinicalTrialId+'.json'
    try:
        with open(CT_PATH) as json_file:
            dataCT = json.load(json_file)
            if DEBUG2:
                print(primaryEvidenceIndexes)
        evidences = collectEvidenceData(dataCT[evidenceSectionId], primaryEvidenceIndexes, clinicalTrialId, evidenceSectionId)
        if DEBUG2:
            print(evidences)
                
    except:
        print('\n!!! Error for file: ' + clinicalTrialId)
    if DEBUG2:
        print("\n")
    return evidences


# preprocessing("/content/dev.json", "dev", "")

## Metrics (F1 score) script

In [25]:
!pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
import numpy
from sklearn.metrics import f1_score,precision_score,recall_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=a89e75bce08f447d110d374014f00c19bff56215d4e1d657fe10d08aa06c9154
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [26]:
import json
import pandas as pd
from functools import reduce 


dev_path = PREFIXG+"dev.json"
with open(dev_path) as json_file:
    dev = json.load(json_file)


def postProcessing(file):
    results2 = {}
    results = {}
    CT_index = 0
    try:
        STM = pd.read_csv(file)
        # print(STM.head(5))
        for id in STM['UUID']:
            prediction = STM['prediction'][CT_index]
            # print(prediction)
            CT_index = CT_index + 1
            if id in results:
              results[id] = results[id] + "," +str(prediction)
            else:
              results[id] = str(prediction)
    except Exception as err:
        print('\n! Error for file')
        print(err)

    for o in results:
        # print(len(results[o]))
        value = {}
        if(len(results[o])>1):
          nums = results[o].split(',')
          preds = list(map(lambda n: int(n), nums))
          # print(preds)
          lst_len= len(preds)
          lst_avg = reduce(lambda x, y: x + y, preds)/lst_len
          if (lst_avg >= 0.5):
            value['Prediction'] = 'Entailment'
          else:
            value['Prediction'] = 'Contradiction'
        else:
          value['Prediction'] = 'Entailment' if int(results[o]) == 1 else 'Contradiction'
        results2[o] = value
    return results2


w = postProcessing(RESULTS_FILE)

def run_score_script(Results):
    gold = dev
    results = Results  
    uuid_list = list(results.keys())

    results_pred = []
    gold_labels = []
    for i in range(len(uuid_list)):
        if results[uuid_list[i]]["Prediction"] == "Entailment":
            results_pred.append(1)
        else:
            results_pred.append(0)
        if gold[uuid_list[i]]["Label"] == "Entailment":
            gold_labels.append(1)
        else:
            gold_labels.append(0)

    f_score = f1_score(gold_labels,results_pred)
    p_score = precision_score(gold_labels,results_pred)
    r_score = recall_score(gold_labels,results_pred)

    print('F1:{:f}'.format(f_score))
    print('precision_score:{:f}'.format(p_score))
    print('recall_score:{:f}'.format(r_score))


Run score script and write results in correct format

In [27]:
run_score_script(w)

with open(RESULTS_FILE_R, 'w') as convert_file:
  convert_file.write(json.dumps(w))

F1:0.607477
precision_score:0.570175
recall_score:0.650000


# Test.json Task 1

In [28]:
def parseEvSg(clinicalTrialId, evidenceSectionId, PREFIX, DEBUG1, DEBUG2):
    evidences = []
    if DEBUG1:
        print("Load premises: " + clinicalTrialId + " " + evidenceSectionId)
    CT_PATH = PREFIX+'CT_data/'+clinicalTrialId+'.json'
    try:
        with open(CT_PATH) as json_file:
            dataCT = json.load(json_file)
        evidences = dataCT[evidenceSectionId]
        # if (len(premises) > 8):
        #   num = 0
        #   for prem in premises:
        #     if (num % 2) == 0:
        #       evidences.append(prem)
        #       num = num + 1
        #       print("PREMISE SELECTIE ----------------------------------")
        # else:
        #   print("PREMISE PUTINE ----------------------------------")
        #   evidences = premises
    except Exception as err:
        print('\n!!! Error for file: ' + clinicalTrialId)
        print(err)
    if DEBUG2:
        print("\n")
    return evidences

In [29]:
TEST_STM = pd.read_csv(TEST_DATA2) #, nrows=40


# exit()
TEST_EVIDENCES=[]
TEST_HYPOTESIS=[]
TEST_UUID=[]
CT_index = -1
for id in TEST_STM['Id']:
    evidences = []
    CT_index = CT_index+1
    CTid = TEST_STM['Primary_id'][CT_index]
    hypotesis = normalizeDocument(TEST_STM['Statement'][CT_index], True)
    hypotesisLen = len(hypotesis.split(' '))
    if (TEST_STM['Type'][CT_index] == "Single"):
      evidences = parseEvSg(CTid, TEST_STM['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      ax = splitEvidence([], "", "[CLS] "+hypotesis+" [SEP] ", evidences, 0, DEBUG2)
      for evidenceFragment in ax:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")
    if (TEST_STM['Type'][CT_index] == "Comparison"):
      CTidSecondary = TEST_STM['Secondary_id'][CT_index]
      premisesPrimary = parseEvSg(CTid, TEST_STM['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      premisesSecondary = parseEvSg(CTidSecondary, TEST_STM['Section_id'][CT_index], PREFIX, DEBUG1, DEBUG2)
      premisesIndexed = splitEvidence([], "", hypotesis, premisesPrimary + premisesSecondary, 0, DEBUG2)
      if (CT_index < 17):
        print(hypotesis)
      for evidenceFragment in premisesIndexed:
        # parse only evidences those that have a minimum of words 
        if(hypotesisLen+len(evidenceFragment) > hypotesisLen + 10):
          TEST_EVIDENCES.append(evidenceFragment)
          TEST_HYPOTESIS.append(hypotesis)
          TEST_UUID.append(id)
        if (CT_index < 17):
          print("     - "+ evidenceFragment)
      # print("-------2--------")
      if DEBUG1:
          print('Analyzed CT entry: ' + str(CT_index) + "  -  "+ CTid+"\n")


women suffering from both claustrophobia and ibs or not eligible for either primary or secondary trial.
     - Inclusion Criteria:   Women are eligible to participate if they are 40 years or older and have been menopausal (had no menstrual bleeding during the past 12 months) Exclusion Criteria:   History of bilateral mastectomy, osteoporosis or renal impairment. Inclusion Criteria: 
     -   Women with a clinically or mammographically identified suspicious breast mass that is likely to be biopsied or surgically removed.Exclusion Criteria:   Pregnancy   Ferromagnetic implants   History of shotgun wounds and shrapnel   Obesity (>250 pounds)   Cardiac pacemaker   Incompatible implanted medical device   Severe claustrophobia 
     -   Major surgeries with potential of ferromagnetic implants  Severe asthma and allergies   i-STAT system, a handheld blood analyzer (I-STAT) creatinine test, estimated glomerular filtration rate (GFR) <30   Metallic object (greater than 2 cm in length) in the br

In [30]:
validation_input = encode_data_to_tensors(TEST_HYPOTESIS, TEST_EVIDENCES, tokenizer, VECTOR_MAX)

# check shape data 
print('DEBUG test shapes: ')
print(validation_input['input_word_ids'].shape)
print(validation_input['input_mask'].shape)
print(validation_input['input_type_ids'].shape)

Dataset MAX (dim): 
99
DEBUG test shapes: 
(1924, 100)
(1924, 100)
(1924, 100)


In [31]:
predictions = [np.argmax(i) for i in model.predict(validation_input)]
# print([i for i in model.predict(test_input)])

submission = pd.DataFrame()
submission['UUID'] = TEST_UUID
submission['prediction'] = predictions

RESULTS_TEST = PREFIXG+"/test-submission-"+stamp_time()+".csv"
RESULTS_TEST_R = PREFIXG+"/test-results-"+stamp_time()+".json"
submission.to_csv(RESULTS_TEST, index = False)
submission.head()

61/61 [==============================] - 13s 219ms/step


,UUID,prediction
0,9f978634-637c-472f-a588-6f4bb2fb121f,0
1,9f978634-637c-472f-a588-6f4bb2fb121f,0
2,9f978634-637c-472f-a588-6f4bb2fb121f,0
3,9f978634-637c-472f-a588-6f4bb2fb121f,0
4,20b34e62-97c2-4ca0-bb1d-7824dab0b8bb,1


In [32]:
submission.to_csv(RESULTS_TEST, index = False)
w = postProcessing(RESULTS_TEST)

with open(RESULTS_TEST_R, 'w') as convert_file:
  convert_file.write(json.dumps(w))